In [1]:
HUGGINGFACE_WORKSPACE="ngohongthai"
DATASET_ID=f"{HUGGINGFACE_WORKSPACE}/dethivaolop6-montoan"
MODEL_ID=f"{HUGGINGFACE_WORKSPACE}/gemma-3N-finetune-dethivaolop6-montoan"

In [2]:
from unsloth import FastModel
model, tokenizer = FastModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = 2048,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/dino/workdpace/learn-llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-21 22:21:21 [__init__.py:235] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.5: Fast Gemma3N patching. Transformers: 4.53.1. vLLM: 0.10.0rc2.dev20+g0ec82edda.
   \\   /|    NVIDIA GeForce RTX 3070. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


Loading checkpoint shards: 100%|██████████| 3/3 [00:14<00:00,  4.74s/it]


In [19]:
def convert_to_prompt(sample):
  if sample["image"] is None:
    return [{
            "role": "user",
            "content": [
                {"type": "text", "text": sample["question"]},
            ],
        }]
  else:
    return [{
            "role": "user",
            "content": [
                {"type": "text", "text": sample["question"]},
                {"type": "image", "image": sample["image"]},
            ],
        }]


In [20]:
from datasets import Dataset, load_dataset
dataset = load_dataset(DATASET_ID, split="test")

In [21]:
prompts = [convert_to_prompt(sample) for sample in dataset]
len(prompts)

95

In [22]:
prompts[0]

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'Tính thể tích hình lập phương có diện tích toàn phần là 96 cm².'},
   {'type': 'image',
    'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=269x260>}]}]

In [23]:
from transformers import TextStreamer
# Helper function for inference
def do_gemma_3n_inference(messages, max_new_tokens = 1024):
    _ = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt = True, # Must add for generation
            tokenize = True,
            return_dict = True,
            return_tensors = "pt",
        ).to("cuda"),
        max_new_tokens = max_new_tokens,
        temperature = 1.0, top_p = 0.95, top_k = 64,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

do_gemma_3n_inference(prompts[0])

<bos><start_of_turn>user
Tính thể tích hình lập phương có diện tích toàn phần là 96 cm².<start_of_image><end_of_turn>
<start_of_turn>model


InductorError: RuntimeError: Failed to run autotuning code block: PY_SSIZE_T_CLEAN macro must be defined for '#' formats

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"
